# SimPy Tutorial: From Basics to Queue Simulation

This tutorial introduces key SimPy concepts and demonstrates how to simulate a basic queue system. You'll learn about:
- **Environment**: The simulation context
- **Process**: Actions that occur over time
- **Resources**: Shared, limited capacity objects
- **Queue Simulation**: Putting it all together

SimPy is a process-based discrete-event simulation framework based on standard Python.

## 1. Installation and Setup

First, let's import the necessary libraries.

In [44]:
import simpy
import random

## 2. SimPy Environment

The **Environment** is the core of SimPy. It manages the simulation time and schedules events.

Let's create a simple environment and run it:

In [45]:
# Create a SimPy environment
env = simpy.Environment()

# Check the current simulation time
print(f"Initial simulation time: {env.now}")

# Run the simulation for 10 time units
env.run(until=10)

print(f"Simulation time after running: {env.now}")

Initial simulation time: 0
Simulation time after running: 10


## 3. Processes

A **Process** is a function that describes events happening over time. Processes use Python's generator functions (with `yield`) to interact with the simulation.

### 3.1 Simple Process with Timeout

Let's create a process that prints messages at different times:

In [46]:
def my_process(env, name, duration):
    """A simple process that waits for a specified duration."""
    print(f"{env.now}: {name} starting")
    
    # Wait for the specified duration
    yield env.timeout(duration)
    
    print(f"{env.now}: {name} finished after {duration} time units")

# Create environment
env = simpy.Environment()

# Start the process
env.process(my_process(env, "Process A", 5))
env.process(my_process(env, "Process B", 3))

# Run simulation
env.run()

0: Process A starting
0: Process B starting
3: Process B finished after 3 time units
5: Process A finished after 5 time units


### 3.2 Multiple Activities in a Process

A process can have multiple activities (timeouts) in sequence:

In [47]:
def car_journey(env, name):
    """Simulate a car journey with multiple stages."""
    print(f"{env.now}: {name} starts driving")
    
    # Drive to gas station
    yield env.timeout(2)
    print(f"{env.now}: {name} arrives at gas station")
    
    # Refuel
    yield env.timeout(1)
    print(f"{env.now}: {name} finishes refueling")
    
    # Drive to destination
    yield env.timeout(3)
    print(f"{env.now}: {name} arrives at destination")

# Create and run simulation
env = simpy.Environment()
env.process(car_journey(env, "Car 1"))
env.run()

0: Car 1 starts driving
2: Car 1 arrives at gas station
3: Car 1 finishes refueling
6: Car 1 arrives at destination


## 4. Resources

A **Resource** represents something with limited capacity that processes need to use (e.g., servers, machines, parking spaces). Processes must request access to a resource and may have to wait if it's busy.

### 4.1 Single Server Resource

In [48]:
def user(env, name, server):
    """A user tries to use a shared server."""
    print(f"{env.now}: {name} arrives and requests server")
    
    # Request the resource
    with server.request() as request:
        # Wait until the resource is available
        yield request
        print(f"{env.now}: {name} got the server")
        
        # Use the resource for some time
        yield env.timeout(2)
        print(f"{env.now}: {name} releases the server")

# Create environment and a single server resource
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)

# Start multiple users
env.process(user(env, "User A", server))
env.process(user(env, "User B", server))
env.process(user(env, "User C", server))

# Run simulation
env.run()

0: User A arrives and requests server
0: User B arrives and requests server
0: User C arrives and requests server
0: User A got the server
2: User A releases the server
2: User B got the server
4: User B releases the server
4: User C got the server
6: User C releases the server


Notice how User B and User C had to wait because the server was busy with the previous user. This is the essence of queueing!

### 4.2 Collecting Statistics

Before building the complete queue, let's learn how to collect statistics during simulation. We'll track wait times by passing a list to our processes:

In [49]:
def user_with_stats(env, name, server, wait_times):
    """A user that records their wait time."""
    arrival_time = env.now
    print(f"{env.now:.2f}: {name} arrives")
    
    with server.request() as request:
        # Wait for the resource
        yield request
        
        # Calculate and record wait time
        wait_time = env.now - arrival_time
        wait_times.append(wait_time)
        print(f"{env.now:.2f}: {name} got server (waited {wait_time:.2f})")
        
        # Use the resource
        yield env.timeout(2)
        print(f"{env.now:.2f}: {name} releases server")

# Create environment and resources
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
wait_times = []  # List to store statistics

# Start multiple users
env.process(user_with_stats(env, "User A", server, wait_times))
env.process(user_with_stats(env, "User B", server, wait_times))
env.process(user_with_stats(env, "User C", server, wait_times))

# Run simulation
env.run()

0.00: User A arrives
0.00: User B arrives
0.00: User C arrives
0.00: User A got server (waited 0.00)
2.00: User A releases server
2.00: User B got server (waited 2.00)
4.00: User B releases server
4.00: User C got server (waited 4.00)
6.00: User C releases server


#### Analyzing Wait Times

Now let's analyze the collected wait times:

In [50]:
print(f"\nCollected wait times: {wait_times}")
print(f"Average wait time: {sum(wait_times) / len(wait_times):.2f}")
print(f"Maximum wait time: {max(wait_times):.2f}")
print(f"Total users served: {len(wait_times)}")


Collected wait times: [0, 2, 4]
Average wait time: 2.00
Maximum wait time: 4.00
Total users served: 3


**Key insight**: By passing a mutable object (like a list) to our processes, we can collect data throughout the simulation and analyze it afterwards. This technique is essential for evaluating system performance.

### 4.3 Custom Resource with Built-in Monitoring

Instead of using a separate monitoring process, we can create a custom resource class that automatically logs utilization. This is an example of extending SimPy's built-in classes:

In [51]:
class MonitoredResource(simpy.Resource):
    """
    A Resource that automatically logs utilization to a stats dictionary.
    
    This class extends simpy.Resource and overrides the request method
    to track utilization data in the same format as other statistics.
    """
    
    def __init__(self, env, capacity=1, stats=None):
        """
        Initialize the monitored resource.
        
        Args:
            env: SimPy environment
            capacity: Number of simultaneous users allowed
            stats: Dictionary to store utilization data (must have 'utilization' key)
        """
        super().__init__(env, capacity)
        self._env = env
        self._capacity = capacity
        self.stats = stats if stats is not None else {'utilization': []}
    
    def request(self, *args, **kwargs):
        """
        Override the request method to log utilization.
        
        Returns:
            Request object from parent class
        """
        # Log current utilization (time, utilization) tuple
        utilization = self.count / self._capacity
        self.stats['utilization'].append((self._env.now, utilization))
        return super().request(*args, **kwargs)

Now let's test the MonitoredResource with a stats dictionary:

In [52]:
def user_monitored(env, name, server, service_time, stats):
    """A user that uses a monitored resource."""
    arrival_time = env.now
    print(f"{env.now:.2f}: {name} arrives")
    
    # Log queue length at arrival
    queue_length = len(server.queue)
    stats['queue_lengths'].append((env.now, queue_length))
    
    with server.request() as request:
        yield request
        
        # Log wait time
        wait_time = env.now - arrival_time
        stats['wait_times'].append(wait_time)
        print(f"{env.now:.2f}: {name} starts service (waited {wait_time:.2f})")
        
        yield env.timeout(service_time)
        print(f"{env.now:.2f}: {name} finishes")

# Create environment with MonitoredResource and stats dictionary
env = simpy.Environment()
stats = {
    'wait_times': [],
    'queue_lengths': [],
    'utilization': []
}
monitored_server = MonitoredResource(env, capacity=1, stats=stats)

# Generate some users with varying service times
random.seed(42)
for i in range(5):
    arrival_time = i * 1.5  # Staggered arrivals
    service_time = random.uniform(1, 3)
    env.process(user_monitored(env, f"User {i+1}", monitored_server, service_time, stats))

# Run simulation
env.run()

0.00: User 1 arrives
0.00: User 2 arrives
0.00: User 3 arrives
0.00: User 4 arrives
0.00: User 5 arrives
0.00: User 1 starts service (waited 0.00)
2.28: User 1 finishes
2.28: User 2 starts service (waited 2.28)
3.33: User 2 finishes
3.33: User 3 starts service (waited 3.33)
4.88: User 3 finishes
4.88: User 4 starts service (waited 4.88)
6.33: User 4 finishes
6.33: User 5 starts service (waited 6.33)
8.80: User 5 finishes


Examine the automatically collected utilization data:

In [53]:
print("\n" + "="*60)
print("MONITORED RESOURCE STATISTICS")
print("="*60)

# Wait Time Statistics
if stats['wait_times']:
    avg_wait = sum(stats['wait_times']) / len(stats['wait_times'])
    print(f"\nWAIT TIME:")
    print(f"  Total customers: {len(stats['wait_times'])}")
    print(f"  Average: {avg_wait:.3f} time units")

# Queue Length Statistics
if stats['queue_lengths']:
    queue_values = [q[1] for q in stats['queue_lengths']]
    avg_queue = sum(queue_values) / len(queue_values)
    print(f"\nQUEUE LENGTH:")
    print(f"  Average: {avg_queue:.2f} customers")
    print(f"  Maximum: {max(queue_values)} customers")

# Utilization Statistics (automatically logged by MonitoredResource)
if stats['utilization']:
    util_values = [u[1] for u in stats['utilization']]
    avg_util = sum(util_values) / len(util_values)
    print(f"\nSERVER UTILIZATION:")
    print(f"  Average: {avg_util:.1%}")
    print(f"  Log entries: {len(stats['utilization'])}")
    print(f"  Sample measurements: {stats['utilization'][:3]}")

print("="*60)


MONITORED RESOURCE STATISTICS

WAIT TIME:
  Total customers: 5
  Average: 3.362 time units

QUEUE LENGTH:
  Average: 1.20 customers
  Maximum: 3 customers

SERVER UTILIZATION:
  Average: 80.0%
  Log entries: 5
  Sample measurements: [(0, 0.0), (0, 1.0), (0, 1.0)]


**Benefits of MonitoredResource with stats dictionary:**

1. **Consistent data structure**: Utilization is logged in the same `stats` dictionary as wait times and queue lengths
2. **Automatic logging**: Utilization is tracked automatically whenever the resource is requested
3. **Encapsulation**: Monitoring logic is contained within the resource class
4. **No separate process needed**: Unlike the `monitor_utilization` function, this approach doesn't require a separate monitoring process
5. **Easy integration**: Works seamlessly with existing statistics collection patterns

This object-oriented approach is cleaner and more maintainable for complex simulations with multiple metrics to track.

## 5. Complete Queue Simulation

Now let's put everything together to create a realistic queue simulation with:
- Random arrival times (Poisson process)
- Random service times (exponential distribution)
- Statistics tracking (wait times, queue length)

### 5.1 Define the Customer Process

In [54]:
def customer(env, name, server, service_time, wait_times):
    """
    Represents a customer arriving, waiting, being served, and leaving.
    
    Args:
        env: SimPy environment
        name: Customer identifier
        server: Server resource
        service_time: Time required for service
        wait_times: List to store wait times
    """
    arrival_time = env.now
    print(f'{arrival_time:.2f}: {name} arrives')
    
    with server.request() as request:
        # Wait in queue
        yield request
        wait_time = env.now - arrival_time
        wait_times.append(wait_time)
        print(f'{env.now:.2f}: {name} starts service (waited {wait_time:.2f})')
        
        # Being served
        yield env.timeout(service_time)
        print(f'{env.now:.2f}: {name} departs')

### 5.2 Define the Customer Generator

This creates a continuous stream of arriving customers:

In [55]:
def customer_generator(env, server, arrival_rate, service_rate, wait_times):
    """
    Generates customers arriving at the queue.
    
    Args:
        env: SimPy environment
        server: Server resource
        arrival_rate: Average arrivals per time unit (lambda)
        service_rate: Average services per time unit (mu)
        wait_times: List to store wait times
    """
    customer_count = 0
    while True:
        # Wait for next arrival (exponentially distributed)
        yield env.timeout(random.expovariate(arrival_rate))
        
        # Create new customer
        customer_count += 1
        service_time = random.expovariate(service_rate)
        env.process(
            customer(env, f'Customer {customer_count}', server, 
                    service_time, wait_times)
        )

### 5.3 Run the Simulation

Let's set up and run the queue simulation:

In [56]:
# Simulation parameters
ARRIVAL_RATE = 1.0      # Average 1 customer per time unit
SERVICE_RATE = 1.5      # Average 1.5 customers served per time unit
SIMULATION_TIME = 20.0  # Run for 20 time units
random.seed(42)         # For reproducibility

# Create environment and resources
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
wait_times = []

# Start the customer generator
env.process(
    customer_generator(env, server, ARRIVAL_RATE, SERVICE_RATE, wait_times)
)

# Run simulation
env.run(until=SIMULATION_TIME)

1.02: Customer 1 arrives
1.02: Customer 1 starts service (waited 0.00)
1.04: Customer 1 departs
1.34: Customer 2 arrives
1.34: Customer 2 starts service (waited 0.00)
1.51: Customer 2 departs
2.68: Customer 3 arrives
2.68: Customer 3 starts service (waited 0.00)
3.43: Customer 3 departs
4.90: Customer 4 arrives
4.90: Customer 4 starts service (waited 0.00)
4.96: Customer 4 departs
5.45: Customer 5 arrives
5.45: Customer 5 starts service (waited 0.00)
5.47: Customer 5 departs
5.70: Customer 6 arrives
5.70: Customer 6 starts service (waited 0.00)
5.72: Customer 7 arrives
6.17: Customer 6 departs
6.17: Customer 7 starts service (waited 0.44)
6.31: Customer 7 departs
6.77: Customer 8 arrives
6.77: Customer 8 starts service (waited 0.00)
7.02: Customer 9 arrives
7.30: Customer 8 departs
7.30: Customer 9 starts service (waited 0.28)
7.89: Customer 9 departs
8.68: Customer 10 arrives
8.68: Customer 10 starts service (waited 0.00)
8.68: Customer 10 departs
10.32: Customer 11 arrives
10.32: Cus

### 5.4 Enhanced Logging with MonitoredResource

Now let's use the `MonitoredResource` class to automatically track server utilization alongside wait times and queue lengths:

In [ ]:
def customer_with_monitoring(env, name, server, service_time, stats):
    """
    Customer process that logs wait times and queue lengths.
    Server utilization is automatically logged by MonitoredResource.
    
    Args:
        env: SimPy environment
        name: Customer identifier
        server: MonitoredResource instance
        service_time: Time required for service
        stats: Dictionary containing lists for different metrics
    """
    arrival_time = env.now
    
    # Log queue length at arrival
    queue_length = len(server.queue)
    stats['queue_lengths'].append((env.now, queue_length))
    
    with server.request() as request:
        # Wait in queue (server.request() automatically logs utilization)
        yield request
        wait_time = env.now - arrival_time
        stats['wait_times'].append(wait_time)
        
        # Being served
        yield env.timeout(service_time)


def customer_generator_with_monitoring(env, server, arrival_rate, 
                                       service_rate, stats):
    """Customer generator for monitored resource simulation."""
    customer_count = 0
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        customer_count += 1
        service_time = random.expovariate(service_rate)
        env.process(
            customer_with_monitoring(env, f'Customer {customer_count}', 
                                    server, service_time, stats)
        )

Run the simulation with MonitoredResource (no separate monitoring process needed):

In [ ]:
# Simulation parameters
ARRIVAL_RATE = 1.0
SERVICE_RATE = 1.5
SIMULATION_TIME = 20.0
random.seed(42)

# Create environment and initialize statistics dictionary
env = simpy.Environment()
stats = {
    'wait_times': [],
    'queue_lengths': [],
    'utilization': []
}

# Create MonitoredResource (automatically logs utilization)
monitored_server = MonitoredResource(env, capacity=1, stats=stats)

# Start customer generator (no separate monitoring process needed!)
env.process(
    customer_generator_with_monitoring(env, monitored_server, 
                                      ARRIVAL_RATE, SERVICE_RATE, stats)
)

# Run simulation
env.run(until=SIMULATION_TIME)

Analyze the comprehensive results:

In [59]:
print("\n" + "="*60)
print("ENHANCED SIMULATION STATISTICS")
print("="*60)

# Wait Time Statistics
if stats['wait_times']:
    avg_wait = sum(stats['wait_times']) / len(stats['wait_times'])
    max_wait = max(stats['wait_times'])
    min_wait = min(stats['wait_times'])
    
    print(f"\nWAIT TIME STATISTICS:")
    print(f"  Total customers served: {len(stats['wait_times'])}")
    print(f"  Average wait time: {avg_wait:.3f} time units")
    print(f"  Maximum wait time: {max_wait:.3f} time units")
    print(f"  Minimum wait time: {min_wait:.3f} time units")

# Queue Length Statistics
if stats['queue_lengths']:
    queue_values = [q[1] for q in stats['queue_lengths']]
    avg_queue = sum(queue_values) / len(queue_values)
    max_queue = max(queue_values)
    
    print(f"\nQUEUE LENGTH STATISTICS:")
    print(f"  Average queue length: {avg_queue:.2f} customers")
    print(f"  Maximum queue length: {max_queue} customers")
    print(f"  Number of measurements: {len(stats['queue_lengths'])}")

# Server Utilization Statistics
if stats['utilization']:
    utilization_values = [u[1] for u in stats['utilization']]
    avg_utilization = sum(utilization_values) / len(utilization_values)
    
    print(f"\nSERVER UTILIZATION STATISTICS:")
    print(f"  Average utilization: {avg_utilization:.1%}")
    print(f"  Server busy: {sum(utilization_values)} measurements")
    print(f"  Server idle: {len(utilization_values) - sum(utilization_values)} measurements")
    print(f"  Total measurements: {len(stats['utilization'])}")

print("="*60)


ENHANCED SIMULATION STATISTICS

WAIT TIME STATISTICS:
  Total customers served: 16
  Average wait time: 0.663 time units
  Maximum wait time: 2.576 time units
  Minimum wait time: 0.000 time units

QUEUE LENGTH STATISTICS:
  Average queue length: 0.44 customers
  Maximum queue length: 2 customers
  Number of measurements: 18

SERVER UTILIZATION STATISTICS:
  Average utilization: 82.5%
  Server busy: 33 measurements
  Server idle: 7 measurements
  Total measurements: 40


**Advantages of using MonitoredResource:**

1. **Wait Times**: Logged manually in the customer process when service begins
2. **Queue Length**: Logged manually at each customer arrival
3. **Server Utilization**: Logged **automatically** by `MonitoredResource` every time `request()` is called

This approach eliminates the need for a separate `monitor_utilization` process, making the code simpler and more maintainable. All three metrics are collected in a unified `stats` dictionary for easy analysis.

### 5.5 Analyze the Results

Let's compute some statistics from our simulation:

In [60]:
print("\n" + "="*50)
print("SIMULATION STATISTICS")
print("="*50)

if wait_times:
    avg_wait = sum(wait_times) / len(wait_times)
    max_wait = max(wait_times)
    min_wait = min(wait_times)
    
    print(f"Total customers served: {len(wait_times)}")
    print(f"Average wait time: {avg_wait:.3f} time units")
    print(f"Maximum wait time: {max_wait:.3f} time units")
    print(f"Minimum wait time: {min_wait:.3f} time units")
    
    # Calculate utilization (rho = lambda / mu)
    utilization = ARRIVAL_RATE / SERVICE_RATE
    print(f"\nTheoretical utilization: {utilization:.1%}")
    print(f"(Server busy {utilization:.1%} of the time)")
else:
    print("No customers were served during the simulation.")

print("="*50)


SIMULATION STATISTICS
Total customers served: 17
Average wait time: 0.081 time units
Maximum wait time: 0.442 time units
Minimum wait time: 0.000 time units

Theoretical utilization: 200.0%
(Server busy 200.0% of the time)


## 6. Key Takeaways

### SimPy Concepts Covered:

1. **Environment (`simpy.Environment()`)**: The simulation context that manages time and events
   - `env.now` - current simulation time
   - `env.run(until=X)` - run simulation until time X
   - `env.process()` - start a new process

2. **Process (generator functions with `yield`)**: Activities that occur over time
   - Use `yield env.timeout(t)` to wait for a duration
   - Use `yield request` to wait for a resource

3. **Resource (`simpy.Resource()`)**: Shared, limited capacity objects
   - `capacity` - number of simultaneous users allowed
   - `request()` - request access to the resource
   - Use `with resource.request() as request:` for automatic release

### Queue Simulation Pattern:

```python
# 1. Create environment
env = simpy.Environment()

# 2. Create shared resources
server = simpy.Resource(env, capacity=1)

# 3. Define process functions with yield
def customer(env, name, server):
    yield env.timeout(...)  # activities
    with server.request() as req:
        yield req  # wait for resource
        yield env.timeout(...)  # use resource

def customer_generator(env, server, arrival_rate, service_rate):
    customer_count = 0
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        customer_count += 1
        service_time = random.expovariate(service_rate)
        env.process(customer(env, f'Customer {customer_count}', 
                            server, service_time))

# 4. Start processes
env.process(customer_generator(env, server, arrival_rate, service_rate))

# 5. Run simulation
env.run(until=TIME)
```

This pattern can be extended to model complex systems with multiple resource types, priorities, interruptions, and more!